# Algorytm FFT

In [2]:
using WAV
using Random
using FFTW

In [3]:
include("naszmodul.jl")

Main.NaszModul

In [4]:
function fft_b(x)
    N = length(x)
    if log2(N) % 1 > 0
        throw(ArgumentError("must be a power of 2"))
    end
    N_min = min(N, 1)
    
    n = collect(0:(N_min-1))
    M = exp.(-2π * im * n * n'/ N_min)
    X = M * reshape(x, (:, N_min))'
    while size(X)[1] < N
        X_even = X[:, 1:Int(size(X)[2] / 2)]
        X_odd = X[:, (Int(size(X)[2] / 2)+1):end]
        t = exp.(-1*im * π * collect(0:(size(X)[1]-1))/size(X)[1])
        terms = hcat(map(x->t, 1:size(X_odd)[2])...)
        X = vcat(X_even + terms .* X_odd, X_even - terms .* X_odd)
    end
    return vec(X)
end
        
        
    

fft_b (generic function with 1 method)

In [219]:
function fft_b2(x)
    N = length(x)
    if log2(N) % 1 > 0
        throw(ArgumentError("must be a power of 2"))
    end
    N_min = min(N, 1)
    
    n = collect(0:(N_min-1))
    M = cis.(-2π * n * n'/ N_min)
    X = M * reshape(x, (:, N_min))'
    while size(X)[1] < N
        X_even = X[:, 1:Int(size(X)[2] / 2)]
        X_odd = X[:, (Int(size(X)[2] / 2)+1):end]
        terms = cis.(-1 * π * collect(0:(size(X)[1]-1))/size(X)[1])
        X = [X_even .+ terms .* X_odd; X_even .- terms .* X_odd]
    end
    return vec(X)
end

fft_b2 (generic function with 1 method)

## Przykład 1

In [220]:
@time p1_n = fft_b([1,2,3,4, 5, 6, 7, 8]);

  0.000115 seconds (145 allocations: 9.078 KiB)


In [221]:
@time p1_o =fft([1,2,3,4, 5, 6, 7, 8]);

  0.000081 seconds (36 allocations: 2.984 KiB)


In [222]:
@time p1_d = NaszModul.dft([1,2,3,4, 5, 6, 7, 8]);

  0.000006 seconds (3 allocations: 560 bytes)


In [223]:
@time p1_ne = fft_b2([1,2,3,4, 5, 6, 7, 8]);

  0.000027 seconds (42 allocations: 4.734 KiB)


In [224]:
isapprox(p1_ne, p1_o)

true

### <span style='color:red'> Klasyczne dft będzie działać zdecydowanie dla długości do 32  </span>

## Przykład 2

In [242]:
p2 = rand(1024);

In [243]:
@time p2_n = fft_b(p2);

  0.001139 seconds (482 allocations: 842.078 KiB)


In [244]:
@time p2_ne = fft_b2(p2);

  0.000470 seconds (100 allocations: 546.922 KiB)


In [136]:
@time p2_o = fft(p2);

  0.000136 seconds (35 allocations: 34.688 KiB)


In [137]:
@time p2_d = NaszModul.dft(p2);

  0.006530 seconds (2 allocations: 32.250 KiB)


In [138]:
isapprox(p2_n, p2_ne)

true

## Przykład 3

In [266]:
p3 = rand(1024^2);

In [267]:
@time p3_n = fft_b(p3);

  1.287454 seconds (1.28 k allocations: 1.508 GiB, 29.17% gc time)


In [268]:
@time p3_ne = fft_b2(p3);

  0.746053 seconds (324 allocations: 1016.016 MiB, 19.80% gc time)


In [145]:
@time p3_o = fft(p3);

  0.101338 seconds (37 allocations: 32.003 MiB, 19.24% gc time)


Dla naszego klasycznego dft końca możemy się nawet nie doczekać

In [24]:
#@time p3_d = NaszModul.dft(p3)

In [256]:
isapprox(p3_n, p3_ne)

true

## Przykład 4

In [260]:
p4 = rand(2^18); #262144

In [261]:
@time p4_n = fft_b(p4);

  0.261783 seconds (1.13 k allocations: 350.033 MiB, 20.20% gc time)


In [262]:
@time p4_ne = fft_b(p4);

  0.246188 seconds (1.13 k allocations: 350.033 MiB, 16.97% gc time)


In [264]:
@time p4_o = fft(p4);

  0.020050 seconds (37 allocations: 8.003 MiB)


In [ ]:
#@time p4_d = NaszModul.dft(p4);

W tym wypadku również klasyczna wersja nie wyrabia za dobrze

In [265]:
isapprox(p4_n, p4_o)

true

## Przykład kolejny (Na razie niewypał)

### <span style='color:green'>Koncepcja dla długości nie będących potęgami liczby 2 - samplowanie lub dopełnianie zerami</span>

In [52]:
bird = wavread("bird.wav")

([0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], 48000.0f0, 0x0010, WAVChunk[WAVChunk(Symbol("fmt "), UInt8[0x10, 0x00, 0x00, 0x00, 0x01, 0x00, 0x02, 0x00, 0x80, 0xbb, 0x00, 0x00, 0x00, 0xee, 0x02, 0x00, 0x04, 0x00, 0x10, 0x00]), WAVChunk(:LIST, UInt8[0x49, 0x4e, 0x46, 0x4f, 0x49, 0x41, 0x52, 0x54, 0x0f, 0x00  …  0x35, 0x38, 0x2e, 0x34, 0x35, 0x2e, 0x31, 0x30, 0x30, 0x00])])

In [53]:
wavplay("bird.wav")

In [54]:
sound = bird[1][:,1]
wind = 1024
len = length(sound)
st = 300
ile_sampli  = (len-wind) ÷ st
slices = Array{Any}(undef,ile_sampli,wind)
for i in 1:ile_sampli
    slices[i, 1:wind] = sound[1+(i-1)*st : wind + (i-1)*st]
end

In [57]:
fft(slices)

LoadError: [91mMethodError: no method matching plan_fft(::Array{Any,2}, ::UnitRange{Int64})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  plan_fft([91m::StridedArray{T, N}[39m, ::Any; flags, timelimit) where {T<:Union{Complex{Float32}, Complex{Float64}}, N} at C:\Users\THINK\.julia\packages\FFTW\G3lSO\src\fft.jl:681[39m
[91m[0m  plan_fft([91m::AbstractArray{var"#s17",N} where N where var"#s17"<:Real[39m, ::Any; kws...) at C:\Users\THINK\.julia\packages\AbstractFFTs\JebmH\src\definitions.jl:199[39m
[91m[0m  plan_fft([91m::AbstractArray{var"#s26",N} where N where var"#s26"<:(Complex{var"#s27"} where var"#s27"<:Union{Integer, Rational})[39m, ::Any; kws...) at C:\Users\THINK\.julia\packages\AbstractFFTs\JebmH\src\definitions.jl:201[39m
[91m[0m  ...[39m

## Algorytm IFFT

In [31]:
function ifft_b(x)
    N = length(x)
    if log2(N) % 1 > 0
        throw(ArgumentError("must be a power of 2"))
    end
    N_min = min(N, 1)
    
    n = collect(0:(N_min-1))
    M = exp.(2π * im * n * n'/ N_min)
    X = M * reshape(x, (:, N_min))'
    while size(X)[1] < N
        X_even = X[:, 1:Int(size(X)[2] / 2)]
        X_odd = X[:, (Int(size(X)[2] / 2)+1):end]
        t = exp.(im * π * collect(0:(size(X)[1]-1))/size(X)[1])
        terms = hcat(map(x->t, 1:size(X_odd)[2])...) #tu chyba coś się pusje
        X = vcat(X_even + terms .* X_odd, X_even - terms .* X_odd)#albo tu
    end
    return (1/N).*[X[1]; reverse(X[2:end])]
end

ifft_b (generic function with 1 method)

## Przykład 5

In [32]:
p5 = fft(rand(16));

In [38]:
@time p5_o = ifft(p5);

  0.000063 seconds (40 allocations: 3.141 KiB)


In [39]:
@time p5_n = ifft_b(p5);

  0.000132 seconds (208 allocations: 16.297 KiB)


In [40]:
@time p5_d = NaszModul.idft(p5);

  0.000026 seconds (2 allocations: 672 bytes)


In [41]:
isapprox(p5_o, p5_n)

true

#### <span style='color:red'> Tak jak w przypadku fft, dla małych długości klasyczna wersja idft będzie szybsza.</span>

## Przykład 6

In [42]:
p6 = fft(rand(1024^2));

In [43]:
@time p5_o = ifft(p6);

  0.094419 seconds (41 allocations: 16.003 MiB, 21.31% gc time)


In [44]:
@time p5_n = ifft_b(p6);

  1.480566 seconds (1.31 k allocations: 1.570 GiB, 15.73% gc time)


Również nasz idft będzie miało duży problem

In [ ]:
#@time p5_d = NaszModul.idft(p6);

In [45]:
isapprox(p5_o, p5_n)

true

## Przykład 7

In [46]:
p7 = fft(rand(2^18)); #ta sama liczba co w przykładzie 4

In [47]:
@time p7_o = ifft(p7);

  0.009599 seconds (41 allocations: 4.003 MiB)


In [48]:
@time p7_n = ifft_b(p7);

  0.533640 seconds (1.16 k allocations: 366.034 MiB, 39.69% gc time)


Tak samo jak wcześniej - idft nie będzie wyrabiał dla tak dużej liczby

In [ ]:
@time p7_d = NaszModul.idft(p7);

In [ ]:
isapprox(p7_o, p7_n)